# Dialogue summarization on DLTK --version 1


This notebook contains a text summarization pipeline using huggingface transformers models.

In this version, we simply utilize available pre-trained models to accomplish dialogue summarization tasks.
Hence, no fine-tuning is involved.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [2]:
# !pip install transformers
# For transformers, tensorflow version>=2.3 is required
import json
import numpy as np
import pandas as pd
# import rrcf as rcf
import transformers
from transformers import pipeline
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [3]:
# print("numpy version: " + np.__version__)
# print("pandas version: " + pd.__version__)
# print("rrcf version: " + rcf.__version__)

numpy version: 1.18.5
pandas version: 1.1.3


## Stage 1 - download transformer model
At stage 1 we choose a suitable model for the summarization task and download the model, which will be used for the runtime.

In [11]:
def get_model(model_name):
    summarizer = pipeline("summarization", model=model_name)
    print("Summarizer initialized.")
    return summarizer

In [12]:
summarizer = get_model("lidiya/bart-base-samsum")

Summarizer initialized.


## Stage 2 - load dialogue transcripts and summarize
At stage 2 we load call dialogues from the client data into a list of call indices and a list of dialogue text strings. 

The summarizer is then called to generate summaries for all the dialogues. 

In [20]:
# Here we define a list of input data (for a local demo) with indices list "index" and dialog text list "dialog"
# This can correspond to real splunk-style work in a real demo

index = ['001','002','003','004','005']
dialog = [
'''
Tim: Hi, what's up? Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating Tim: What did you plan on doing? Kim: Oh you know, uni stuff and unfucking my room Kim: Maybe tomorrow I'll move my ass and do everything Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores Tim: It really helps Kim: thanks, maybe I'll do that Tim: I also like using post-its in kaban style
''',
'''
Sarah: I found a song on youtube and I think you'll like it James: What song? Sarah: <file_other> James: Oh. I know it! James: I heard it before in some compilation Sarah: I can't stop playing it over and over James: That's exactly how I know lyrics to all of the songs on my playlist :D Sarah: Haha. No lyrics here though. Instrumental ;D James: Instrumental songs are different kind of music. James: But you have to remember that the activity you do when you listen to this song James: Is the actvity your brain will connect to the song James: And everytime you play this song at home James: You'll be thinking of your work Sarah: Yeah, I know that. That's why we sometimes say - I used to like that song, but now it just reminds me of bad memories James: Yup. Everytime you change your partner, you have to get rid of your favorite music :D Sarah: Hahaha. True, true.
''',
'''
Leon: did you find the job yet? Arthur: no bro, still unemployed :D Leon: hahaha, LIVING LIFE Arthur: i love it, waking up at noon, watching sports - what else could a man want? Leon: a paycheck? ;) Arthur: don't be mean... Leon: but seriously, my mate has an offer as a junior project manager at his company, are you interested? Arthur: sure thing, do you have any details? Leon: <file_photo> Arthur: that actually looks nice, should I reach out directly to your friend or just apply to this email address from the screenshot? Leon: it's his email, you can send your resume directly and I will mention to him who you are :)
''',
'''
Elisa: Who wants to come for drinks tonight at Mombasa? Alice: me! Sadie: I'm in as well Carol: may I bring some people? Elisa: sure, the more the better! Carol: amazing Arthur: count me in! I'll come with Alice of course Liam: me too! Kai: I will join you but after my courses, around 8 Elisa: No problem, we'll be there till midnight at least Tom: i may come over :) Elisa: Anybody else? I'd like to book the corner table Tom: the big one? Elisa: exactly John: So count me in as well Elisa: Perfect, the table is for 15 people so we should manage! See you there
''',
'''
Gabriella: Hey Jasmine, how are you doing? Jasmine: I’m fine, thanks 😊 Gabriella: Would you do me a favor? Jasmine: No problem. How can I help? Gabriella: I need to send my CV in English to a truly perfect job for me and I’m wondering if you could take a look at it? Jasmine: Sure! Send it to me 😊 Gabriella: Done 😊 Jasmine: Got it. Give me an hour and I’ll check it Gabriella: Perfect! Thank you soooo much! Jasmine: I’ve checked it. I must say that there wasn’t much to correct. It was practically perfect. I just fixed a little bit grammar and a few spelling mistakes. Gabriella: I know how great you are at spelling! Jasmine: Thanks Gabriella. I hope that you will get that job! Gabriella: I hope so, too. Jasmine: I’ll keep my fingers crossed 😊
'''
]       
    

In [21]:
results = summarizer(dialog,max_length=60)

## Stage 3 - assign summaries to their indices
In this stage the summarization results are indiced and written to an output file

In [22]:
ret = {}
for i, result in enumerate(results):
    ret[index[i]] = result['summary_text']

In [23]:
ret

{'001': 'Kim was going to do lots of stuff but ended up procrastinating. Tim recommends Pomodoro technique where Kim uses breaks for doing chores.',
 '002': 'Sarah found a song on youtube and James thinks she will like it.',
 '003': "Arthur's mate has an offer as junior project manager at his company. Arthur will apply to Leon's friend's email address.",
 '004': 'Elisa, Alice, Sadie, Carol, Arthur, Liam and Kai will join Elisa for drinks tonight at Mombasa. Elisa will book a corner table for 15 people.',
 '005': 'Jasmine needs to send her CV in English to a truly perfect job for her. Gabriella will take a look at it.'}

Next step: save "ret" into Splunk database to be used for analysis or visualizations.